In [1]:
# Dependencies
import pandas as pd 
import json
from config import token
import requests
import time

In [2]:
# creating list of dates with month interval
start = "2005-01-01"
end = "2020-06-01"
month_list = [i.strftime("%Y-%m-%d") for i in pd.date_range(start=start, end=end, freq='MS')]

from_date = []
to_date = []

# looping thru the list of dates and appending to and from dates
for index, date in enumerate(month_list):
    if index == 0:
        from_date.append(date)
    else:
        if index == -1:
            to_date.append(date)
        else:
            from_date.append(date)
            to_date.append(date)

In [3]:
#function to create url
def create_url(from_date, to_date, token):
    url = 'https://finnhub.io/api/v1/calendar/ipo?from={}&to={}&token={}'.format(
        from_date, # date format 2000-01-01
        to_date,   # date format 2000-01-01
        token
    )
    return url

In [4]:
def ipo_data(from_date, to_date, token):
    print("Beginning Data Retrieval")
    print("-------------------------------")
    
    ipos_list = []
    
    for from_dt, to_dt in zip(from_date, to_date):
        
        print (f'Attempting to get IPO data for {from_dt}-{to_dt}')

        url = create_url(from_dt, to_dt, token)
        response = requests.get(url)
        result = response.json()['ipoCalendar']

        for item in result:
            date = item['date']
            exchange = item['exchange']
            name = item['name']
            numberOfShares = item['numberOfShares']
            price = item['price']
            status = item['status']
            symbol = item['symbol']
            totalSharesValue = item['totalSharesValue']

            ipos_list.append([(date,
                             exchange,
                             name,
                             numberOfShares,
                             price,
                             status,
                             symbol,
                             totalSharesValue)])
        time.sleep(5)

    ipo_df = pd.DataFrame([item for ipo_list in ipos_list for item in ipo_list])
    ipo_df.columns = ['Date',
                     'Exchange',
                     'Company Name',
                     'Number of Shares',
                     'Price',
                     'Status',
                     'Symbol',
                     'Number of Shares'
                        ]
    
    print("Data Retrieval Complete")
    return ipo_df

In [5]:
ipo_df = ipo_data(from_date,to_date,token)

Beginning Data Retrieval
-------------------------------
Attempting to get IPO data for 2005-01-01-2005-02-01
Attempting to get IPO data for 2005-02-01-2005-03-01
Attempting to get IPO data for 2005-03-01-2005-04-01
Attempting to get IPO data for 2005-04-01-2005-05-01
Attempting to get IPO data for 2005-05-01-2005-06-01
Attempting to get IPO data for 2005-06-01-2005-07-01
Attempting to get IPO data for 2005-07-01-2005-08-01
Attempting to get IPO data for 2005-08-01-2005-09-01
Attempting to get IPO data for 2005-09-01-2005-10-01
Attempting to get IPO data for 2005-10-01-2005-11-01
Attempting to get IPO data for 2005-11-01-2005-12-01
Attempting to get IPO data for 2005-12-01-2006-01-01
Attempting to get IPO data for 2006-01-01-2006-02-01
Attempting to get IPO data for 2006-02-01-2006-03-01
Attempting to get IPO data for 2006-03-01-2006-04-01
Attempting to get IPO data for 2006-04-01-2006-05-01
Attempting to get IPO data for 2006-05-01-2006-06-01
Attempting to get IPO data for 2006-06-01-

Attempting to get IPO data for 2017-11-01-2017-12-01
Attempting to get IPO data for 2017-12-01-2018-01-01
Attempting to get IPO data for 2018-01-01-2018-02-01
Attempting to get IPO data for 2018-02-01-2018-03-01
Attempting to get IPO data for 2018-03-01-2018-04-01
Attempting to get IPO data for 2018-04-01-2018-05-01
Attempting to get IPO data for 2018-05-01-2018-06-01
Attempting to get IPO data for 2018-06-01-2018-07-01
Attempting to get IPO data for 2018-07-01-2018-08-01
Attempting to get IPO data for 2018-08-01-2018-09-01
Attempting to get IPO data for 2018-09-01-2018-10-01
Attempting to get IPO data for 2018-10-01-2018-11-01
Attempting to get IPO data for 2018-11-01-2018-12-01
Attempting to get IPO data for 2018-12-01-2019-01-01
Attempting to get IPO data for 2019-01-01-2019-02-01
Attempting to get IPO data for 2019-02-01-2019-03-01
Attempting to get IPO data for 2019-03-01-2019-04-01
Attempting to get IPO data for 2019-04-01-2019-05-01
Attempting to get IPO data for 2019-05-01-2019

In [6]:
ipo_df.head()

,Date,Exchange,Company Name,Number of Shares,Price,Status,Symbol,Number of Shares
0,2005-02-01,None,OAK STREET FINANCIAL SERVICES INC,NaN,None,withdrawn,None,NaN
1,2005-01-31,None,TRANSPORTATION TECHNOLOGIES INDUSTRIES INC,NaN,None,withdrawn,None,NaN
2,2005-01-28,New York Stock Exchange,W&T OFFSHORE INC,12655263.0,19.00,priced,WTI,240449997.0
3,2005-01-28,Nasdaq National Market,DFC GLOBAL CORP.,7500000.0,16.00,priced,DLLR,120000000.0
4,2005-01-28,Nasdaq National Market,NATIONAL INTERSTATE CORP,3847000.0,13.50,priced,NATL,51934500.0


In [8]:
ipo_df.to_csv("output/ipo_calendar.csv", index=False)